## Finance Scape

In [31]:
# Modules
from selenium import webdriver
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
# from selenium.common.exceptions import TimeoutException
from selenium import webdriver
import chromedriver_binary  # Adds chromedriver binary to path
from selenium.webdriver.chrome.options import Options
from coinbase.wallet.client import Client

# Log-in Credentials
from Credentials import *

# Formatting
from money import Money
import smtplib
from email.message import EmailMessage
import time



class Scraper:
    
    def __init__(self,credentials):
        self.credentials = credentials
        
    def waitforxpath(self,driver,path): 
        try:
            element = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH,path)))
        finally:
            s=driver.find_element(By.XPATH, path)
        return s
            
    def waitforname(self,driver,name): 
        try:
            element = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.NAME,name)))
        finally:
            s=driver.find_element(By.NAME, name)
            time.sleep(0.5)
        return s
    
    # Money Dashboard
    def MD(self):
        chrome_options = Options()  
        chrome_options.add_argument("--headless")  
        driver = webdriver.Chrome(options=chrome_options)
        driver.get('https://my.moneydashboard.com/?signin=true')

        self.waitforxpath(driver,"/html/body/div[4]/md-dialog/md-dialog-actions/button").click()
        self.waitforname(driver,"email").send_keys(credentials["md_email"])
        self.waitforname(driver,"password").send_keys(credentials["md_password"])
        self.waitforxpath(driver,"/html/body/ui-view/div[2]/div[1]/div/form/div/div[1]/div[2]/button").click()
        self.waitforxpath(driver,'//*[@id="navbar"]/div[2]/div[1]/md-icon').click()
        
        time.sleep(2)
        value =self.waitforxpath(driver,'//*[@id="navbar"]/div[3]/div[1]/div[1]/div[1]').text
        return value

    
    # Hargreaves Lansdown
    def HL(self):
        chrome_options = Options()  
        chrome_options.add_argument("--headless")  
        driver = webdriver.Chrome(options=chrome_options)
        driver.get('https://online.hl.co.uk/my-accounts/login-step-one')

        self.waitforxpath(driver,'//*[@id="username"]').send_keys(credentials["HL_username"])
        self.waitforxpath(driver,'//*[@id="date-of-birth"]').send_keys(credentials["HL_dob"])
        self.waitforxpath(driver,'//*[@id="login"]/p[1]/input').click()
        self.waitforxpath(driver,'//*[@id="online-password-verification"]').send_keys(credentials["HL_password"])

        secure_number_input1 = driver.find_element_by_name('secure-number[1]')
        secure_number_input2 = driver.find_element_by_name('secure-number[2]')
        secure_number_input3 = driver.find_element_by_name('secure-number[3]')

        val1=int(secure_number_input1.get_attribute("title")[10])-1
        val2=int(secure_number_input2.get_attribute("title")[10])-1
        val3=int(secure_number_input3.get_attribute("title")[10])-1

        secure_number_input1.send_keys(credentials["HL_secure_number"][val1])
        secure_number_input2.send_keys(credentials["HL_secure_number"][val2])
        secure_number_input3.send_keys(credentials["HL_secure_number"][val3])

        self.waitforxpath(driver,'//*[@id="submit"]').click()
        time.sleep(2)
        value = driver.find_elements_by_xpath('//*[@id="content-body-full"]/div/div[2]/table/tfoot/tr/td[2]')[0].text
        return value

    
    def CB(self):
        client = Client(credentials["C_api_key"], credentials["C_api_secret"])
        total = 0
        message = []
        accounts = client.get_accounts()
        for wallet in accounts.data:
            message.append( str(wallet['name']) + ' ' +   str(wallet['native_balance']) )
            value = str( wallet['native_balance']).replace('GBP','')
            total += float(value)
        message.append( 'Total Balance: ' + 'GBP ' + str(total) )
        #print('\n'.join( message ))
        return total
    
    def email(self,text):
        gmail_user = credentials["Gmail"]
        gmail_password = credentials["Gpassword"]
        
        try:
            server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
            server.ehlo()
            server.login(gmail_user, gmail_password)
            msg = EmailMessage()
            msg.set_content(text)
            msg['Subject'] = f'Personal Finance'
            msg['From'] = gmail_user
            msg['To'] = credentials["Gto"]
            server.send_message(msg)
            server.quit()
        
        except:
            print('Email will not send')

    def formatting(self,md,hl,cb):
        md1 = Money(amount=md.replace(',', '')[1:], currency='GBP')
        hl1 = Money(amount=hl.replace(',', '')[1:], currency='GBP')
        cb1 = Money(amount=cb, currency='GBP')
        total = (md1+hl1+cb1)
        print(md1)
        print(hl1)
        print(cb1)
        print(total)
        text = "Santander Accounts = "+ md1.format('en_US', '¤#,##0.00')     \
                +"\nHargreaves Lansdown = "+ hl1.format('en_US', '¤#,##0.00') \
                +"\nCoinbase = "+ cb1.format('en_US', '¤#,##0.00')             \
                +"\n\nTotal = "+total.format('en_US', '¤#,##0.00')
        return text
            
    def run_code(self):
        money_dashboard = self.MD()
        hargreaves_lansdown = self.HL()
        coinbase = self.CB()
        
        body=self.formatting(money_dashboard,hargreaves_lansdown,coinbase)
        self.email(body)
        
    

instance=Scraper(credentials)
instance.run_code()

GBP 7,248.30
GBP 1,231.57
GBP 1,156.38
GBP 9,636.25
